In [44]:
import numpy as np
import pandas as pd
import geopandas as gpd
import psrcelmerpy

In [103]:
eg_conn = psrcelmerpy.ElmerGeoConn()

# Load network shapefile
gdf = gpd.read_file(r'R:\e2projects_two\SoundCast\Inputs\dev\networks\2023\network_2023_v3\shapefiles\AM\AM_edges.shp')

In [104]:
# Load bike stress table, adapted from Lowry
# https://www1.coe.neu.edu/~pfurth/Furth%20papers/2016%20Prioritizing%20to%20improve%20low-stress%20network%20connectivity%20Lowry,%20Furth,%20Hadden-Loh.pdf

df_data = pd.read_csv('bike_stress_table.csv')

In [105]:
# Reformat speed limit and lanes to match the bike stress data
# Creating a new column for speed limits where minimum is capped at 25 and max at 35
gdf['SpeedLimit_new'] = gdf['ul2'].copy()
gdf.loc[gdf['ul2'] <= 25, 'SpeedLimit_new'] = 25
gdf.loc[gdf['ul2'] >= 35, 'SpeedLimit_new'] = 35

# Set maximum lanes at 6 and minimum at 2

gdf['lanes_new'] = gdf['lanes'].copy().astype('int')
gdf.loc[gdf['lanes'] >= 6, 'lanes_new'] = 6
gdf.loc[gdf['lanes'] <2, 'lanes_new'] = 2

# Lowry has different values for neighborhood street for each bike facility type
# Create a flag for an local street
gdf['LocalStreet'] = 0
gdf.loc[gdf['FacilityTy'] == 9, 'LocalStreet'] = 1
# gdf['FacilityType'].unique()

In [108]:
gdf['FacilityTy'] = gdf['FacilityTy'].astype('int')
gdf['FacilityTy'].unique()

# Remove centroid connectors and freeways
facility_type_lookup = {
    1:'Freeway',   # Interstate
    2:'Freeway',   # Other Freeway
    3:'Freeway', # Expressway
    4:'Ramp',
    5:'Arterial',    # Principal arterial
    6:'Arterial',    # Minor Arterial
    7:'Collector',    # Major Collector
    8:'Collector',    # Minor Collector
    9:'Collector',   # Local
    10:'Busway',
    11:'Non-Motor',
    12:'Light Rail',
    13:'Commuter Rail',
    15:'Ferry',
    16:'Passenger Only Ferry',
    17:'Connector',    # centroid connector
    18:'Connector',    # facility connector
    19:'HOV',    # HOV Only Freeway
    20:'HOV'    # HOV Flag
    }

gdf = gdf[gdf['FacilityTy'].isin([5,6,7,8,9,11])]

In [109]:
len(gdf)

61011

In [110]:
# Intersect with urban growth area
gdf_growth_area = eg_conn.read_geolayer('urban_growth_area')
gdf_growth_area = gdf_growth_area[['county_name','geometry']].to_crs('EPSG:2285')

gdf = gpd.sjoin(gdf, gdf_growth_area, how="left")

gdf['rural'] = 'not_rural'
gdf.loc[gdf['county_name'].isnull(),'rural'] = 'rural'

In [112]:
# Define transRefEdges facility types to match the study definitions

bike_type_map = {
    0: 'NoBikeFacility',    # No Bike Lane
    1: 'BikeLane',    # Striped Bike Lane
    2: 'ProtectedBikeLane',    # Protected Bike Lane
    3: 'NoBikeFacility',    # Paved Shoulder
    4: 'Sharrows',    # Shared Lane Markings
    5: 'NoBikeFacility',    # Bike Provision Undefined
    6: 'NoBikeFacility',    # Bike Provision Undefined
    8: 'SharedUsePath',    # Shared Use Path
    9: 'BufferedBikeLane',    # Buffered Bike Lane 
    10: 'NoBikeFacility',     # Neighborhood Greenway
    
}

# Create a mapping of local vs other area type
area_type_map = {
    0: 'Other',
    1: 'Neighborhood'
}

# Create new fields that use the bike facility and area type mapping
gdf['BikeFacility'] = gdf['bkfac'].map(bike_type_map)
gdf['AreaType'] = gdf['LocalStreet'].map(area_type_map)

# We will follow this designation for any local street with speeds 30 mph or less and 2 lanes (1 in each direction)
# If speed limit > 30, set area type to non-neighborhood
gdf.loc[(gdf['SpeedLimit_new']>30, 'AreaType')] = 'Other'

# If number of lanes > 2, set area type to non-neighborhood
# FIXME: should probably just be 1 lane in each direction
gdf.loc[(gdf['lanes_new']>2, 'AreaType')] = 'Other'

# If in rural area, treat paved shoulder as bike lanes
gdf.loc[((gdf['rural']=='rural')&(gdf['bkfac']==3)), 'BikeFacility'] = 'BikeLane'

# Merge to df_data to get stress values
gdf = gdf.merge(df_data, left_on=['SpeedLimit_new', 'lanes_new', 'BikeFacility','AreaType'], 
          right_on=['SpeedLimit', 'Lanes', 'BikeFacility', 'AreaType'], how='left')

In [114]:
# add impact of slope
gdf.loc[(gdf['upslp'] > .02) & (gdf['upslp'] <= .04), 'Factor'] = gdf['Factor'] + 0.37
gdf.loc[(gdf['upslp'] > .04) & (gdf['upslp'] <= .06), 'Factor'] = gdf['Factor'] + 1.2
gdf.loc[(gdf['upslp'] > .06), 'Factor'] = gdf['Factor'] + 3.2

In [115]:
# Assign LTS based on stress factor
gdf.loc[gdf['Factor'] < 0.1, 'LTS'] = 1
gdf.loc[(gdf['Factor'] >= 0.1) & (gdf['Factor'] < 0.3), 'LTS'] = 2
gdf.loc[(gdf['Factor'] >= 0.3) & (gdf['Factor'] < 0.6), 'LTS'] = 3
gdf.loc[gdf['Factor'] >= 0.6, 'LTS'] = 4

In [116]:
# Create new column names because ArcGIS is a dinosaur that can't handle more than 10 character headers
gdf.rename(columns={'BikeFacility':'bkfac', 'SpeedLimit':'speed'}, inplace=True)

In [117]:
gdf['ij'] = gdf['i'].astype('str')+'-'+gdf['j'].astype('str')
gdf['ji'] = gdf['j'].astype('str')+'-'+gdf['i'].astype('str')

gdf['ji_exists'] = 0
gdf.loc[gdf['ji'].isin(gdf['ij']), 'ji_exists'] = 1

# Get a ji dataframe
gdf_ji = gdf[gdf['ji_exists']==1].copy()
gdf_ji.drop(['ij','geometry'], axis=1, inplace=True)
gdf_ji.rename(columns={'ji': 'ij'}, inplace=True)

# # merge that to gdf
gdf_new = gdf.copy()
gdf_new = gdf_new[['ij','upslp',  'bkfac','Lanes', 'speed', 'Factor','LTS','geometry','PSRCEdgeID']].merge(gdf_ji[['ij','upslp', 'bkfac','Lanes', 'speed', 'Factor', 'LTS',]],
                                                                                      how='left', on='ij', suffixes=['_IJ','_JI'])
len(gdf_new)

61285

In [122]:
# Get a single record for each Edge ID
gdf_new = gdf_new.groupby('PSRCEdgeID').first().reset_index()
# gdf_new

In [123]:
# Merge with WSDOT and ped notebooks (created in separate notebooks)
gdf_ped = gpd.read_file('2023_ped_stress_index.shp')
gdf_wsdot = gpd.read_file('2023_network_bike_stress_wsdot.shp')

In [126]:
gdf_merge = gdf_new.merge(gdf_wsdot[['ij','BTLS_IJ','BTLS_JI']], on='ij', how='left')
gdf_merge = gdf_merge.merge(gdf_ped[['ij','PTLS_IJ','PTLS_JI']], on='ij', how='left')

In [127]:
gdf_merge.rename(columns={'LTS_IJ':'LTS_IJ_psrc', 'LTS_JI':'LTS_JI_psrc',
                          'BTLS_IJ': 'LTS_IJ_wsdot', 'BTLS_JI': 'LTS_JI_wsdot'}, inplace=True)

In [131]:
# gdf_merge.to_file('2023_bike_ped_stress.shp')
gdf_merge.to_csv(r'T:\2024November\brice\2023_bike_ped_stress_11_05_24.csv')

In [141]:
gdf_merge['ij'] = gdf_merge['ij'].astype('str')
gdf_merge = gdf_merge.set_crs('EPSG:2285')

In [142]:
gdf_merge.to_file(r'T:\2024November\brice\2023_bike_ped_stress_11_05_24.shp')

c:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [143]:
# gdf_merge

In [85]:
# gdf_merge = gpd.read_file(r'T:\2024November\brice\2023_bike_ped_stress_11_04_24.shp')

In [130]:
# Merge model network back to the pedbike inventoru

gdf_pedbike = gpd.read_file(r'C:\Workspace\2023PedBikeInventory_062624_Cleaned\2023PedBikeInventory_062624_Cleaned.shp')

In [95]:
gdf_pedbike.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [96]:
gdf_merge = gdf_merge.set_crs('EPSG:2285')
gdf_merge = gdf_merge.to_crs(gdf_pedbike.crs)
# gdf_pedbike = gdf_pedbike.to_crs('EPSG:2285')


In [97]:
gdf_test = gpd.sjoin(gdf_pedbike, gdf_merge, how='left')

In [101]:
gdf_test[~gdf_test['PTLS_IJ'].isnull()]

,OBJECTID_1,Enabled,PSRCEdgeID_left,dateLastUp,ActiveLink,LinkType,Fullname,DateCreate,LastEditor,Source,...,upslp_JI,bkfac_JI,Lanes_JI,speed_JI,Factor_JI,LTS_JI_psr,LTS_IJ_wsd,LTS_JI_wsd,PTLS_IJ,PTLS_JI
1,498148,1,183993,2024-06-17,1,90,12TH,2018-01-26,KOverby,None,...,25.0,NoBikeFacility,2.0,139370.0,0.2,2.0,3.0,1.0,2.0,1.0
1,498148,1,183993,2024-06-17,1,90,12TH,2018-01-26,KOverby,None,...,30.0,NoBikeFacility,2.0,137975.0,3.6,4.0,3.0,3.0,2.0,2.0
3,464381,1,150226,2022-10-12,1,90,RAINIER,2018-01-26,scoe,None,...,30.0,NoBikeFacility,2.0,119762.0,0.4,3.0,3.0,4.0,2.0,3.0
4,464391,1,150236,2021-12-06,1,90,GRADY,2018-01-26,koverby,None,...,25.0,NoBikeFacility,2.0,124170.0,0.2,2.0,3.0,2.0,2.0,1.0
11,479748,1,165593,2021-02-26,3,90,21ST,2018-01-26,koverby,None,...,35.0,NoBikeFacility,2.0,31429.0,1.0,4.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37280,501793,1,187638,2021-02-26,1,90,7TH,2018-01-26,koverby,None,...,25.0,NoBikeFacility,2.0,77529.0,0.2,2.0,3.0,3.0,2.0,2.0
37283,501802,1,187647,2021-02-26,1,90,50TH,2018-01-26,koverby,None,...,25.0,NoBikeFacility,2.0,75203.0,0.2,2.0,3.0,2.0,2.0,1.0
37285,509188,1,195033,2021-02-26,1,90,SR 99,2018-01-26,koverby,None,...,25.0,NoBikeFacility,2.0,71738.0,0.2,2.0,3.0,1.0,2.0,1.0
37296,501940,1,187785,2021-02-26,1,90,SHILSHOLE,2018-01-26,koverby,None,...,25.0,SharedUsePath,2.0,198688.0,0.0,1.0,1.0,1.0,1.0,1.0
